# Case Study 6
Pulling from examples found from 6.5 rather than the original from 6 which is plagued with issues.

# Imports

# Dataset

In [ ]:
def clean_text(text):
    # Replace multiple newlines with two newlines (preserving paragraph breaks)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    # Replace multiple spaces with a single space
    text = re.sub(r' +', ' ', text)
    return text.strip()

In [ ]:
# Lazily read the file as a CSV with one column ("line"), treating each line as a row.
df = pl.scan_csv(
    "/content/full_books_text.txt",
    has_header=False,
    new_columns=["line"],
    separator="\n",            # Use newline as the record separator
    ignore_errors=True,        # Skip rows that cause parsing errors
    infer_schema_length=10000, # Increase inference length for schema detection
    schema_overrides={"line": pl.Utf8},
    quote_char=None            # Disable quoting so no fields are interpreted as quoted
)

# Now, split each line into tokens (splitting on whitespace) and explode the tokens.
tokens_df = df.with_columns(
    pl.col("line").str.split(" ").alias("tokens")
).explode("tokens")

# Collect the first 100 tokens.
first_100_tokens = tokens_df.select("tokens").head(100).collect()["tokens"].to_list()

# Count the total number of tokens.
total_tokens = tokens_df.select(pl.count("tokens")).collect().item()

# Count the unique tokens.
unique_tokens = tokens_df.select(pl.col("tokens").n_unique()).collect().item()

print("First 100 tokens:")
print(" ".join(first_100_tokens))
print("\nToken counts:")
print("Total tokens:", total_tokens)
print("Unique tokens:", unique_tokens)

In [ ]:
# Collect in streaming mode to avoid memory spikes.
limited_tokens = tokens_df_limited.select("tokens").collect(streaming=True)["tokens"].to_list()

# In case the list has None values, filter them out.
limited_tokens = [token for token in limited_tokens if token is not None]

# Then take the first 100 tokens.
first_100_tokens = limited_tokens[:100]

print("First 100 tokens:")
print(" ".join(first_100_tokens))